In [ ]:
"""

當用戶想要重新觀看歡迎訊息時，可輸入特定文字，Server會重新寄發歡迎訊息

當用戶發送指定文字消息時，會發送相應的圖片消息給用戶

"""

In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/food" , static_folder = "../food/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
server_url

'0b71b5b8.ngrok.io'

In [2]:
secretFileContentJson.get("channel_access_token")

'YGQ8o7Y8naocftGCq0M4XKSzF7XoSl4EyrsjUOqmxcYoGXndEELUdC5Q5RlJXyIoL8IRtN/3YaxjcXkZJMBSbqt82yGSJjD885hForRc7zBFpn6RE3EmPw2TNd7KRKrg+l0T9ZzHNGoxWuD1jh4BKwdB04t89/1O/w1cDnyilFU='

In [3]:
# MySQL套件
import pymysql

# 將QRcode上傳MySQL資料庫
conn = pymysql.connect(
    host = "uuboyscymysql.clrhltpp3icl.ap-northeast-1.rds.amazonaws.com",
    port = int(3306),user = "food",
    password = "food",
    db = "food",
    charset='utf8', 
    cursorclass=pymysql.cursors.DictCursor 
    )
cursor = conn.cursor()
print('Connected successfully!')

Connected successfully!


In [4]:
'''

製作文字與圖片發送消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 設定消息素材
text_reply_message1 = TextSendMessage(text="歡迎光臨無人商店")
text_reply_message2 = TextSendMessage(text="更多資訊請點選下列選單")
'''image_reply_message1 = ImageSendMessage(
                                        original_content_url='https://%s/images/003.jpeg' %server_url ,
                                        preview_image_url='https://%s/images/001.jpg' %server_url)
image_reply_message2 = ImageSendMessage(
                                        original_content_url='https://%s/images/004.png' %server_url,
                                        preview_image_url='https://%s/images/005.jpg' %server_url)'''

'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="歡迎光臨無人超市"),
    TextSendMessage(text="更多資訊請點選下列選單")
]

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息
4. 製作該用戶的QRcode
5. 將QRcode同時保存於資料庫及'../food'
6. 同時將userID上傳至資料庫

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests

# 製作QRcode的程式
import create_qrcode


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../foodusers.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    print()
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 製作該使用者QRcode
    create_qrcode.create_qrcode(user_profile.user_id, user_profile.user_id)
    
    # 將使用者userID, name, qrcode 上傳至MySQL資料庫
    f = open('../food/%s.jpg'%(user_profile.user_id), 'rb')
    qrcode = f.read()
    f.close()
    try:
        cursor.execute("INSERT INTO membersInfo SET userID=%s, name=%s, qrcode=%s", 
                       (str(user_profile.user_id), 
                        str(user_profile.display_name), 
                        qrcode))
        conn.commit()
        # 回覆文字消息與圖片消息
        line_bot_api.reply_message(
            event.reply_token,[
                TextSendMessage(text="歡迎光臨NobodyMart無人超市，感謝您成為NobodyMart會員！"),
                TextSendMessage(text="尊敬的 %s 會員您好！"%(user_profile.display_name)),
                TextSendMessage(text="您的USER_ID是 " + user_profile.user_id),
                TextSendMessage(text="您的QR_code，進入超市前請先出示QR_code驗證身份"),
                ImageSendMessage(original_content_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id),
                                 preview_image_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id)),
                TextSendMessage(text="更多資訊請點選下列選單"),       
            ]
        )
    except:
        line_bot_api.reply_message(
            event.reply_token,[
                TextSendMessage(text="Hi～ %s ，歡迎回到NobodyMart無人超市！"%(user_profile.display_name)),
                TextSendMessage(text="您的USER_ID是 " + user_profile.user_id),
                TextSendMessage(text="您的QR_code，進入超市前請先出示QR_code驗證身份"),
                ImageSendMessage(original_content_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id),
                                 preview_image_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id)),
                TextSendMessage(text="更多資訊請點選下列選單"),       
            ]
        )
    #print('userID existed.')
    


In [5]:
'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    ImageSendMessage, TextSendMessage
)

# 圖片消息的基本建構特徵
image_message = TextSendMessage(text='2019-01-23\n雞肉(1)($ 80)\n麵粉(1)($ 50)\n------------------\nTotal  $130')
image_message2 = TextSendMessage(text='暫無紀錄')
image_message3 = TextSendMessage(text='此功能未實裝')

image_message4 = TextSendMessage(text='不能改哦～這功能放好玩的！')

In [6]:
'''

設計一個Carousel Flex
引入carousel的python檔
引入交易紀錄查詢的python程式

'''
# 食譜推薦
import recipe_recommanded_carousel

# 會員中心
import member_center

# 食譜推薦
flexCarouselContainerJsonDict = recipe_recommanded_carousel.flexCarouselContainerJsonDict

# 會員中心

# 交易紀錄
import transaction_history

In [7]:
'''

將carousel類型的json 進行轉換變成 Python可理解之類型物件

將該物件封裝進 Flex Message中


'''

from linebot.models import(
    FlexSendMessage,CarouselContainer,BubbleContainer
)

import json

carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
flexCarouselSendMeesage =  FlexSendMessage(alt_text="hello", contents=carouselContent)

In [8]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "[::text:]本次交易":image_message,
    "[::text:]歷史交易":image_message2,
    "[::flex:]促銷食材":flexCarouselSendMeesage,
    "[::text:]會員中心":image_message3,
    "[::text:]more":image_message3,
    "[::text:]Modify":image_message4,
    "[::text:]tryLiff":image_message3
    #"[::text:]教學訊息":[text_reply_message1,text_reply_message2,image_reply_message1,image_reply_message2]
}

In [9]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''
from random import randint

# 用來將過多的Liff刪除
from liff_api import *
# initiate token
liff_init(secretFileContentJson.get("channel_access_token"))

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # to remove leftover liff
    try:
        if len(liff_list()['apps']) >= 15:
            for i in liff_list()['apps']:
                liff_delete(i['liffId'])
    except:
        print('Liffs have been all removed.')
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    # 更改為該會員資訊
    qrcodeurl = 'https://%s/food/%s.jpg' %(server_url, user_profile.user_id)
    cursor.execute("SELECT name FROM membersInfo WHERE userID='%s'"%(user_profile.user_id))
    
    username = (cursor.fetchone()['name']).encode('utf-8').decode('unicode-escape')
    flexCarouselContainerJsonDict_member_center = member_center.member_centre_flex(user_profile.user_id, server_url, qrcodeurl, (user_profile.user_id), username)    
    try:
        carouselContent_member_center = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict_member_center))
    except json.decoder.JSONDecodeError:
        print('json.decoder.JSONDecodeError in handle_message.')
        username = 'User name error please modify'
        flexCarouselContainerJsonDict_member_center = member_center.member_centre_flex(user_profile.user_id, server_url, qrcodeurl, (user_profile.user_id), username)
        carouselContent_member_center = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict_member_center))
    flexCarouselSendMeesage_member_center =  FlexSendMessage(alt_text="hello", contents=carouselContent_member_center)
    
    template_message_dict["[::text:]會員中心"] = flexCarouselSendMeesage_member_center
        
    # Try Liff
    flexBubbleContainerJsonString_transaction_data = transaction_history.tryLiff()
    bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
    flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="try", contents=bubbleContainer_transaction_data)
    template_message_dict["[::text:]tryLiff"] = flexBubbleSendMessage_transaction_data
    
    # 透過MySQL查詢歷史交易
    cursor.execute("SELECT datetime FROM transaction WHERE userID = '%s' GROUP BY orderID"%(user_profile.user_id))
    transaction_data = cursor.fetchall()
    # 將交易紀錄資料送入flex轉換的transaction_history.py python程式
    flexBubbleContainerJsonString_transaction_data = transaction_history.transaction_flex(user_profile.user_id, server_url, transaction_data)
    bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
    flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="hello", contents=bubbleContainer_transaction_data)
    template_message_dict["[::text:]歷史交易"] = flexBubbleSendMessage_transaction_data
    
    # 本次即時交易紀錄
    # 將UserID送入flex轉換的transaction_history.py python程式
    flexBubbleContainerJsonString_transaction_data = transaction_history.transaction_now_flex(user_profile.user_id, server_url)
    bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
    flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="hello", contents=bubbleContainer_transaction_data)
    template_message_dict["[::text:]本次交易"] = flexBubbleSendMessage_transaction_data
    
    if(event.message.text.find('::text:')!= -1) or (event.message.text.find('::flex:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    elif(event.message.text.find('白痴')!= -1) or (event.message.text.find('笨蛋')!= -1) or (event.message.text.find('智障')!= -1) or (event.message.text.find('神經病')!= -1) or (event.message.text.find('有病')!= -1) or (event.message.text.find('王八蛋')!= -1) or (event.message.text.find('傻蛋')!= -1) or (event.message.text.find('台灣人')!= -1) :
        tmpstr = ''
        if event.message.text.find('笨蛋')!= -1:
            tmpstr = '笨蛋'
        if event.message.text.find('白痴')!= -1:
            tmpstr = '白痴'
        if event.message.text.find('智障')!= -1:
            tmpstr = '智障'
        if event.message.text.find('神經病')!= -1:
            tmpstr = '神經病'
        if event.message.text.find('有病')!= -1:
            tmpstr = '有病'
        if event.message.text.find('王八蛋')!= -1:
            tmpstr = '王八蛋'
        if event.message.text.find('傻蛋')!= -1:
            tmpstr = '傻蛋'
        if event.message.text.find('台灣人')!= -1:
            tmpstr = '台灣人'
        if randint(0,4) == 1:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你才%s！'%(tmpstr))
            )
        elif randint(0,4) == 2:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='%s才會說別人是%s！'%(tmpstr, tmpstr))
            )
        elif randint(0,4) == 3:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='今天天氣真好！')
            )
        elif randint(0,4) == 4:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你媽貴姓？！')
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你全家都%s！'%(tmpstr))
            )
    elif (event.message.text.find('幹你娘')!= -1) or (event.message.text.find('靠北')!= -1) or (event.message.text.find('靠背')!= -1) or (event.message.text.find('操你')!= -1) or (event.message.text.find('操他媽')!= -1) or (event.message.text.find('靠被')!= -1) or (event.message.text.find('幹他媽')!= -1) or (event.message.text.find('幹你')!= -1) or (event.message.text.find('靠杯')!= -1):
        if randint(0,4) == 1:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你確定？')
            )
        elif randint(0,4) == 2:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='這樣不太好吧？')
            )
        elif randint(0,4) == 3:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='今天天氣真好！')
            )
        elif randint(0,4) == 4:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你媽貴姓？！')
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='讓我考慮考慮')
            )
    else:
        '''line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=["請點擊功能選單，或輸入[::text:]more，取得更多幫助"+str(event.message.text)+"test", "second"])
        )'''
        line_bot_api.reply_message(
        event.reply_token,
        {'%s'%(str(event.message.text)) : [TextSendMessage(text='%s'%(str(event.message.text).replace('?', '!').replace('？', '！').replace('嗎','').replace('我','你'))), TextSendMessage(text='點擊下方功能選單，取得更多幫助。')],
        '更多幫助':image_message3,
        '更多資訊':image_message3,
        'more':image_message3}.get(event.message.text)
        )
        '''{"%s"%(str(event.message.text)) : ['123', 'test', '456']}.get(event.message.text)'''

In [10]:
'''

查詢歷史交易紀錄之接口

'''

@app.route('/transaction/<userID>/<datetime>', methods = ['GET'])
def transactionInfo(userID, datetime):
    # transaction query
    cursor.execute("SELECT tr.orderID, pt.productName, tr.productQTY, pt.productUnitPrice, tr.productQTY*pt.productUnitPrice subtotal FROM transaction tr \
    LEFT JOIN productTable pt \
    ON tr.productID = pt.productID \
    WHERE userID = '%s' AND datetime='%s'"%(userID, datetime))
    trInfo = cursor.fetchall()
    # total
    cursor.execute("SELECT SUM(tr.productQTY*pt.productUnitPrice) total FROM transaction tr \
    LEFT JOIN productTable pt \
    ON tr.productID = pt.productID \
    WHERE userID = '%s' AND datetime='%s'"%(userID, datetime))
    trTotal = cursor.fetchall()
    
    # head
    outStr = """
    <html>
    <head>
    <title>Order_ID : %s</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    <link href="https://%s/food/web_js_css/flexslider.css" rel="stylesheet" type="text/css" />
    </head>
    """%(trInfo[0]['orderID'], server_url)
    # body
    outStr += """
    <body>
    <script src="https://%s/food/web_js_css/jquery.min.js"></script>
    <script src="https://%s/food/web_js_css/jquery.flexslider.js"></script>
    <div>
    <div class="col-lg-9 col-lg-offset-9">
    """%(server_url, server_url)
    # body-table
    outStr += """
    <table class="table" style="font-size:30px;font-family:serif;">
                <thead>
                    <tr class="danger">
                        <th>No.</th>
                        <th>Product Name</th>
                        <th>Quantity</th>
                        <th>Unit Price</th>
                        <th>Subtotal</th>
                    </tr>
                </thead>    
    """
    # tbody
    outStr += """
    <tbody>
    """
    for i in range(0, len(trInfo)):
        tcolor = ["success", "warning"]
        outStr += """
        <tr class="%s">
            <td>%s</td>
            <td>%s</td>
            <td>%s</td>
            <td>$ %s</td>
            <td>$ %s</td>
        </tr>
        """%(tcolor[i%2], i+1, trInfo[i]['productName'], trInfo[i]['productQTY'], trInfo[i]['productUnitPrice'], trInfo[i]['subtotal'])
    outStr += """
    <thead>
        <tr class="danger">
        <th>Total</th>
        <th>-</th>
        <th>-</th>
        <th>-</th>
        <th>$ %s</th>
        </tr>
    </thead>
    """%(trTotal[0]['total'])
    outStr += """
    </tbody>
    </table>
    """
    # carousel
    outStr += """
    <br><br>
    <div style="margin:0 auto;"><h1><p align=center>-  On-sale products  -</p><h1></div>
    <div class="container">
    <script type="text/javascript">
    $(window).load(function() {
    $('.flexslider').flexslider({
          slideshowSpeed: 5000,
          animationSpeed: 500,
          animation: "slide",
          touch: true
          });
    });
    </script>
    <div class="flexslider" style="width:450px;  margin:0 auto;">
    <ul class="slides">
        <li><img src="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2018/05/29/draft/4804805.jpeg&x=0&y=0&sw=0&sh=0&exp=3600" alt="" width="350" height="200"></li>
        <li><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgszxK4wWxfhfCokY5PwlL52bzi6FTa7vILrrewvOPW6-jDeYGzg" alt="" width="350" height="200"></li>
        <li><img src="https://www.foodnext.net/dispPageBox/getFile/GetImg.aspx?FileLocation=/PJ-FOODNEXT/Files/&FileName=photo-08083-i.jpg" alt="" width="350" height="200"></li>
    </ul>
    </div>
    </div>
    """

    # logo image
    outStr += """
    <div class="container">
    <img class="img-responsive" src="https://%s/food/logo.png">
    </div>
    """%(server_url)
    # /body
    outStr += """
    </div>
    </div>
    </body>
    </html>
    """
    
    return outStr


In [11]:
'''

查詢當下交易紀錄之接口

'''

@app.route('/transaction/now/<userID>', methods = ['GET'])
def transactionNow(userID):
    # head
    outStr = """
    <html>
    <head>
    <title>User_ID : %s</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    <link href="https://%s/food/web_js_css/flexslider.css" rel="stylesheet" type="text/css" />
    </head>
    """%(userID, server_url)
    # body
    outStr += """
    <body>
    <script src="https://%s/food/web_js_css/jquery.min.js"></script>
    <script src="https://%s/food/web_js_css/jquery.flexslider.js"></script>
    <div>
    <div class="col-lg-9 col-loutStrrrset-9">
    """%(server_url, server_url)
    # body-table
    outStr += """
    <table class="table" style="font-size:30px;font-family:serif;">
                <thead>
                    <tr class="danger">
                        <th>No.</th>
                        <th>Product Name</th>
                        <th>Quantity</th>
                        <th>Unit Price</th>
                        <th>Subtotal</th>
                    </tr>
                </thead>    
    """
    # tbody
    outStr += """
    <tbody>
    """
    
    tcolor = ["success", "warning"]
    outStr += """
    <tr class="success">
        <td>1</td>
        <td>Salt</td>
        <td>1</td>
        <td>$ 40</td>
        <td>$ 40</td>
    </tr>
    <tr class="warning">
        <td>2</td>
        <td>Egg</td>
        <td>3</td>
        <td>$ 30</td>
        <td>$ 90</td>
    </tr>
    <tr class="success">
        <td>3</td>
        <td>Chicken</td>
        <td>1</td>
        <td>$ 150</td>
        <td>$ 150</td>
    </tr>
    <tr class="warning">
        <td>4</td>
        <td>OliveOil</td>
        <td>1</td>
        <td>$ 120</td>
        <td>$ 120</td>
    </tr>
    """
    outStr += """
    <thead>
        <tr class="danger">
        <th>Total</th>
        <th>-</th>
        <th>-</th>
        <th>-</th>
        <th>$ 400</th>
        </tr>
    </thead>
    """
    
    # /tbody /table
    outStr += """
    </tbody>
    </table>
    """
    
    # carousel
    outStr += """
    <br><br>
    <div style="margin:0 auto;"><h1><p align=center>-  On-sale products  -</p><h1></div>
    <div class="container">
    <script type="text/javascript">
    $(window).load(function() {
    $('.flexslider').flexslider({
          slideshowSpeed: 5000,
          animationSpeed: 500,
          animation: "slide",
          touch: true
          });
    });
    </script>
    <div class="flexslider" style="width:450px;  margin:0 auto;">
    <ul class="slides">
        <li><img src="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2018/05/29/draft/4804805.jpeg&x=0&y=0&sw=0&sh=0&exp=3600" alt="" width="350" height="200"></li>
        <li><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgszxK4wWxfhfCokY5PwlL52bzi6FTa7vILrrewvOPW6-jDeYGzg" alt="" width="350" height="200"></li>
        <li><img src="https://www.foodnext.net/dispPageBox/getFile/GetImg.aspx?FileLocation=/PJ-FOODNEXT/Files/&FileName=photo-08083-i.jpg" alt="" width="350" height="200"></li>
    </ul>
    </div>
    </div>
    """
    
    # logo image
    outStr += """
    <div class="container">
    <img class="img-responsive" src="https://%s/food/logo.png">
    </div>
    """%(server_url)
    # /body
    outStr += """
    </div>
    </div>
    </body>
    """
    
    return outStr

In [12]:
'''

會員中心資料修改

'''

@app.route('/usercentre/<userID>', methods=['GET'])
def getdata(userID):
    outStr = """
    <!doctype html>
    <html>
    <head>   
    
    <title>USER CENTRE</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    </head>
    
    <body>
    <script type="text/javascript">
    function ValidateValue(textbox) {
        var IllegalString = "[`~!#$^&*()=|{}':;',\\[\\].<>/?~！#￥……&*（）——|{}【】‘；：”“'。，、？]‘'";
        var textboxvalue = textbox.value;
        var index = textboxvalue.length - 1;
        var s = textbox.value.charAt(index);
        if (IllegalString.indexOf(s) >= 0) {
        s = textboxvalue.substring(0, index);
        textbox.value = s;
        }
    }
    </script>
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
        <thead>
        <th>
        Hello, your name is ?
        </th>
        </thead>
        </table>
    <p></p>
    <table class="table" style="font-family:serif;margin:0 auto;">
    <thead>
    <th>
        <form action="/usercentre/%s" method="post">
         <label>Name:</label>
         <input type="textbox" name="username" onkeyup = "ValidateValue(this)">
         <input type="submit" value="Submit">
        </form>
    </th>
    </thead>
    </table>
    </div>
    </body>
    </html>
    """%(userID)
    #return render_template('text.html')
    return outStr

@app.route('/usercentre/<userID>', methods=['POST'])
def submit(userID):
    name = request.form.get('username')
    print(name)
    print('Find \\ ', ('\\' in name), ' , in submit')
    
    content = """
    Please try a shorter name.
    <br>
    <input type="button" value="Return" onclick="location.href='https://%s/usercentre/%s'">
    """%(server_url, userID)
    
    if '\\' in name:
        name = "ErrorTyping"
        content = """
        Please type regular words (\).
        <br>
        <input type="button" value="Return" onclick="location.href='https://%s/usercentre/%s'">
        """%(server_url, userID)
    elif len(name.encode('unicode-escape').decode('utf-8').replace('\\','\\\\')) < 39 :
        content = """Your user name has been modified to <br>%s !"""%(name)
        cursor.execute("UPDATE membersInfo SET name='%s' WHERE userID='%s'"%(name.encode('unicode-escape').decode('utf-8').replace('\\','\\\\'), userID))
        conn.commit()    
    
    outStr = """
    <html>
    <head>   
    
    <title>USER CENTRE</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    </head>
    
    <body>
    
    <table class="table" style="font-family:serif;margin:0 auto;">
    <thead>
    <th>
    %s
    </th>
    </thead>
    </table>
    
    </body>
    </html>
    """%(content)
    return outStr

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.160.4 - - [12/Feb/2019 02:59:47] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 02:59:50] "GET /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 03:00:05] "POST /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -


\ga
Find \  True  , in submit


192.168.160.4 - - [12/Feb/2019 03:00:14] "GET /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 03:00:21] "POST /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -


施丞優
Find \  False  , in submit


140.115.236.39 - - [12/Feb/2019 03:10:22] "GET /hello/?token=name HTTP/1.1" 404 -
192.168.160.4 - - [12/Feb/2019 03:24:49] "GET /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 03:32:09] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 03:32:10] "POST / HTTP/1.1" 200 -


...

/opt/conda/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (1300, "Invalid utf8 character string: '89504E'")
  self._do_get_result()
/opt/conda/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (1366, "Incorrect string value: '\\xE7\\xA7\\x89\\xE9\\xB4\\xBB' for column 'name' at row 1")
  self._do_get_result()
192.168.160.4 - - [12/Feb/2019 09:25:32] "POST / HTTP/1.1" 200 -
191.97.37.217 - - [12/Feb/2019 09:35:41] "GET / HTTP/1.1" 405 -
192.168.160.4 - - [12/Feb/2019 09:38:41] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 09:39:09] "GET /food/U5dc989d65f1897bd44fcbd415b47e164.jpg HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 09:54:52] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 09:55:04] "GET /transaction/U7173dc299af6ac3fe7697a75c2a280ad/2019-02-05%2002:30:42 HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:03:50] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:03:55] "GET /transaction/now/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1

\
Find \  True  , in submit


192.168.160.4 - - [12/Feb/2019 10:18:22] "GET /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:18:44] "GET /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:18:55] "POST /usercentre/U7173dc299af6ac3fe7697a75c2a280ad HTTP/1.1" 200 -


UUBOY
Find \  False  , in submit


192.168.160.4 - - [12/Feb/2019 10:19:00] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:19:34] "POST / HTTP/1.1" 200 -
192.168.160.4 - - [12/Feb/2019 10:19:36] "GET /food/logo.png HTTP/1.1" 200 -
166.62.123.125 - - [12/Feb/2019 11:06:27] "GET / HTTP/1.1" 405 -
